In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [4]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%


torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [5]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [7]:
batch_size = 32
n = batch_size
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [8]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1) * (bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits- logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3377, grad_fn=<NegBackward0>)

In [64]:
bnraw.shape , bndiff.shape , bnvar_inv.shape , bndiff.shape , bnmeani.shape , hprebn.shape

(torch.Size([32, 64]),
 torch.Size([32, 64]),
 torch.Size([1, 64]),
 torch.Size([32, 64]),
 torch.Size([1, 64]),
 torch.Size([32, 64]))

In [ ]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

# 1) cmp('logprobs', dlogprobs, logprobs)
dlogprobs = torch.zeros_like(logprobs)  
dlogprobs[range(n), Yb] = -1.0 / n      # dL/dlogprobs = -1/n
cmp('logprobs', dlogprobs, logprobs)

#2)  cmp('probs', dprobs, probs)

dprobs = torch.zeros_like(probs)
dprobs[range(n), Yb] = -1/n * (1/(probs[range(n), Yb])) # dL/dprobs = -1/n * (1/(probs))
# -1/n is out gradient as in autograd and (1/(probs)) is the local gradient
cmp('probs', dprobs, probs)

#3 ) cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

# Note : 
# dcounts_sum_inv is the gradient of loss w.r.t the inverse normalization factor (1 / counts_sum).
# Negative value means increasing counts_sum_inv (i.e., decreasing counts_sum) lowers the loss.
# Decreasing counts_sum shrinks the softmax denominator, increasing normalized probabilities.
# Thus, the loss decreases when the normalization factor grows, boosting the true class probabilities.

dcounts_sum_inv = torch.zeros_like(counts_sum_inv)
# counts = [32, 27] and dprobs = [32, 27] and counts_sum_inv = [32, 1] so gradient of counts_sum_inv (dcounts_sum_inv) should be [32, 27]
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True) 
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

# 4) cmp('counts_sum', dcounts_sum, counts_sum)
 
dcounts_sum  = torch.zeros_like(counts_sum)
dcounts_sum  = - dcounts_sum_inv * (counts_sum**-2)
cmp('counts_sum', dcounts_sum, counts_sum)

# 5) cmp('counts', dcounts, counts)

dcounts = torch.zeros_like(counts)
# (Indirect term gradient ) + (Direct term gradient)
dcounts = 1 * dcounts_sum  + counts_sum_inv * dprobs
cmp('counts', dcounts, counts)

# 6) cmp('norm_logits', dnorm_logits, norm_logits)

dnorm_logits = dcounts * counts
cmp('norm_logits', dnorm_logits, norm_logits)

# 7) cmp('logit_maxes', dlogit_maxes, logit_maxes)

# only if the maximum logit value changes then the 
dlogit_maxes = -1 * dnorm_logits.sum(1, keepdim=True) 
cmp('logit_maxes', dlogit_maxes, logit_maxes)

# 8) cmp('logits', dlogits, logits)

# Small difference due to floating point precision
dlogits  = torch.zeros_like(logits)
max_idx = logits.argmax(1, keepdim=True)
# Indirect term gradient
dlogits[range(n), max_idx] = 1.0 # gradient of the max value is 1 else 0 
dlogits *= dlogit_maxes
# Now Direct term gradient
dlogits += 1 * dnorm_logits
cmp('logits', dlogits, logits)

# 9) cmp('h', dh, h)
# thats the only way we can achieve the shape (see vid derivation for it )
dh = torch.zeros_like(h)
dh = dlogits @ W2.T
cmp('h', dh, h)

# 10) cmp('W2', dW2, W2)
dW2 = torch.zeros_like(W2)
dW2 = h.T @ dlogits
cmp('W2', dW2, W2)

# 11) cmp('b2', db2, b2)

# only way to get b2 shape 
db2 = torch.zeros_like(b2)
db2 = dlogits.sum(0, keepdim=True)
cmp('b2', db2, b2)

# 12) cmp('hpreact', dhpreact, hpreact)
dhpreact = torch.zeros_like(hpreact)
dhpreact = (1.0 - h**2 ) * dh
cmp('hpreact', dhpreact, hpreact)

# 13) cmp('bngain', dbngain, bngain)
dbngain = torch.zeros_like(bngain)
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
cmp('bngain', dbngain, bngain)

# 14) cmp('bnbias', dbnbias, bnbias)
dbnbias = (1*dhpreact).sum(0, keepdim=True)
cmp('bnbias', dbnbias, bnbias)

# 15) cmp('bnraw', dbnraw, bnraw)
dbnraw = (bngain * dhpreact) 
cmp('bnraw', dbnraw, bnraw)

# 16) cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)

# 17) cmp('bnvar', dbnvar, bnvar)
dbnvar = -0.5 * (bnvar + 1e-5)**(-1.5) * dbnvar_inv
cmp('bnvar', dbnvar, bnvar)

# 18) cmp('bndiff2', dbndiff2, bndiff2)

dbndiff2= 1/(n-1) * dbnvar
cmp('bndiff2', dbndiff2, bndiff2)

# 19) cmp('bndiff', dbndiff, bndiff)
# (Indirect gradient )
dbndiff = 2 * bndiff * dbndiff2
# (direct gradient )
dbndiff += bnvar_inv * dbnraw
cmp('bndiff', dbndiff, bndiff)

#20) cmp('bnmeani', dbnmeani, bnmeani)
dbnmeani = -1 * dbndiff.sum(0, keepdim=True)
cmp('bnmeani', dbnmeani, bnmeani)

#21) cmp('hprebn', dhprebn, hprebn)

# bnmeani = 1/n * hprebn.sum(0, keepdim=True)
# hprebn -> [a11 a12
#          a21 a22]  

#  bnmeani -> [m1 , m2] = 1/n [ a11 + a21   a12 + a22 ]
#                           1/n [ 1   1 
#                                 1   1 ]

# (Indirect gradient )
dhprebn = dbndiff.clone() # or just dbndiff
# (Direct gradient )
dhprebn += (1.0/n) * (torch.ones_like(hprebn) * dbnmeani)
cmp('hprebn', dhprebn, hprebn)


# 22) cmp('embcat', dembcat, embcat)
dembcat = dhprebn @ W1.T
cmp('embcat', dembcat, embcat)

# 23) cmp('W1', dW1, W1)
dW1 = embcat.T @ dhprebn
cmp('W1', dW1, W1)

# 24) cmp('b1', db1, b1)
db1 = dhprebn.sum(0) # as b1.shape is [n_hidden,]
cmp('b1', db1, b1)

# 25) cmp('emb', demb, emb)
demb = dembcat.view(emb.shape) # dembcat is embcat.view(emb.shape)
cmp('emb', demb, emb)

# 26) cmp('C', dC, C)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
  for j in range(Xb.shape[1]):
    ix = Xb[k,j]
    dC[ix] += demb[k,j]
    
# cmp('logprobs', dlogprobs, logprobs)
# cmp('probs', dprobs, probs)
# cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
# cmp('counts_sum', dcounts_sum, counts_sum)
# cmp('counts', dcounts, counts)
# cmp('norm_logits', dnorm_logits, norm_logits)
# cmp('logit_maxes', dlogit_maxes, logit_maxes)
# cmp('logits', dlogits, logits)
# cmp('h', dh, h)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0


RuntimeError: The size of tensor a (27) must match the size of tensor b (32) at non-singleton dimension 1

torch.Size([32, 3, 10])

In [13]:
demb.shape

torch.Size([32, 3, 10])